In [0]:
SELECT current_catalog() AS catalog, current_schema() AS schema;

In [0]:
SHOW CATALOGS;

In [0]:
SHOW SCHEMAS IN workspace;


In [0]:
CREATE SCHEMA IF NOT EXISTS workspace.bronze;


In [0]:
SHOW TABLES IN workspace.default;


In [0]:
ALTER TABLE workspace.default.device_messages
RENAME TO workspace.bronze.device_messages_raw;

ALTER TABLE workspace.default.rapid_step_tests
RENAME TO workspace.bronze.rapid_step_tests_raw;


In [0]:
SHOW TABLES IN workspace.bronze;


In [0]:
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows
FROM workspace.bronze.device_messages_raw
UNION ALL
SELECT 'rapid_step_tests_raw', COUNT(*)
FROM workspace.bronze.rapid_step_tests_raw;

In [0]:
SELECT
  device_id,
  COUNT(*) AS message_count
FROM workspace.bronze.device_messages_raw
GROUP BY device_id
ORDER BY message_count DESC;


In [0]:
DESCRIBE TABLE workspace.bronze.rapid_step_tests_raw;


In [0]:
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  to_timestamp(from_unixtime(test_time / 1000))  AS test_ts,
  total_steps,
  size(step_points) AS step_points_len
FROM workspace.bronze.rapid_step_tests_raw
LIMIT 20;


In [0]:
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  test_time AS test_time_raw,
  total_steps,
  size(step_points) AS step_points_len
FROM workspace.bronze.rapid_step_tests_raw
LIMIT 20;


In [0]:
CREATE OR REPLACE VIEW workspace.bronze.rapid_step_tests_clean AS
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  test_time                                     AS test_time_raw,
  step_points,
  total_steps,
  token
FROM workspace.bronze.rapid_step_tests_raw;


In [0]:
DESCRIBE TABLE workspace.bronze.device_messages_raw;

In [0]:
SELECT *
FROM workspace.bronze.device_messages_raw
LIMIT 10;

In [0]:
SELECT
  device_id,
  date,
  to_timestamp(from_unixtime(date / 1000))      AS date_ts_guess,
  timestamp,
  to_timestamp(from_unixtime(timestamp / 1000)) AS ts_ts_guess,
  message
FROM workspace.bronze.device_messages_raw
LIMIT 25;

In [0]:
CREATE OR REPLACE VIEW workspace.bronze.device_messages_clean AS
SELECT
  message_origin,
  session_key,
  message,
  device_id,
  sensor_type,
  distance,
  date AS date_raw,
  timestamp AS timestamp_raw,
  to_timestamp(from_unixtime(date / 1000)) AS event_ts
FROM workspace.bronze.device_messages_raw;


In [0]:
SELECT device_id, event_ts, message, sensor_type, distance
FROM workspace.bronze.device_messages_clean
ORDER BY event_ts DESC
LIMIT 20;


In [0]:
SELECT
  r.customer,
  r.device_id,
  r.start_ts,
  r.stop_ts,
  r.total_steps,
  d.event_ts,
  d.sensor_type,
  d.distance,
  d.message
FROM workspace.bronze.rapid_step_tests_clean r
JOIN workspace.bronze.device_messages_clean d
  ON r.device_id = d.device_id
 AND d.event_ts BETWEEN r.start_ts AND r.stop_ts
LIMIT 50;


In [0]:
CREATE OR REPLACE VIEW workspace.bronze.step_test_device_messages AS
SELECT
  r.customer,
  r.device_id,
  r.start_ts,
  r.stop_ts,
  r.total_steps,
  r.step_points,
  d.event_ts,
  d.sensor_type,
  d.distance,
  d.message_origin,
  d.session_key,
  d.message
FROM workspace.bronze.rapid_step_tests_clean r
JOIN workspace.bronze.device_messages_clean d
  ON r.device_id = d.device_id
 AND d.event_ts BETWEEN r.start_ts AND r.stop_ts;
